In [1]:

from my_modules.log import log,Limit,CounterBreak,CounterMod


from my_modules.load_data import MyDataset,ConcatDataset
import my_modules.model.lstm as model_modules
from my_modules.model.lstm import LSTM_0,move_to_gpu,del_gpu,LSTM_1,LSTM_2,LSTM_3,LSTM_4

from my_modules.model.lstmcnn_part3 import LSTMCNN_10,LSTMCNN_10_2
from my_modules.model.lstm2cnn_part1 import LSTM2CNN_1,LSTM2CNN_2,LSTM2CNN_3,LSTM2CNN_4
from my_modules.model.lstm2cnn_final import LSTM2CNN,LSTM2CNN_2



import torch
from torch import nn
from torch.utils.data import DataLoader

import os
# os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

from tqdm import tqdm

print(model_modules.get_used_memory())

from playsound import playsound
sound_path = r'C:\Users\rakna\Desktop\AIT_working\AIT_CP_Project\code\notice_sound'




0


In [2]:
train_index_movie = [i for i in range(38)]
test_index_movie = [i for i in range(len(train_index_movie),40)]


people_name = []
people_name = [*people_name,*[f's0{i}.dat' for i in range(1,9+1)]]
people_name = [*people_name,*[f's{i}.dat' for i in range(10,32+1)]]
data_root_path = r'C:\Users\rakna\Desktop\AIT_working\AIT_CP_Project\this_folder_git_ignore\ori_datasets'

data_path = [os.path.join(data_root_path,path) for path in people_name]



# window_size = 145*5
window_size = 145*10
step_window = 10

list_train_dataset = [MyDataset(_path,window_size,step_window,train_index_movie,do_noise=True) for _path in data_path]
list_test_dataset = [MyDataset(_path,window_size,step_window,test_index_movie) for _path in data_path]


data_train = ConcatDataset(*list_train_dataset)
data_test = ConcatDataset(*list_test_dataset)


print(data_train.len_of_dataset)
print(len(data_train)) #804992
print(data_train[50000])


[25156, 25156, 25156, 25156, 25156, 25156, 25156, 25156, 25156, 25156, 25156, 25156, 25156, 25156, 25156, 25156, 25156, 25156, 25156, 25156, 25156, 25156, 25156, 25156, 25156, 25156, 25156, 25156, 25156, 25156, 25156, 25156]
804992
(tensor([[ 2.6753e+00,  8.1925e+00,  1.8993e+01,  ..., -2.5978e+01,
         -1.2039e+01, -3.2680e+00],
        [-1.2657e+01, -2.5779e+01, -4.6467e+01,  ...,  6.1634e+01,
          2.8586e+01, -1.3370e+00],
        [-3.5147e+00, -3.1367e+00,  2.4314e+00,  ..., -5.2270e+00,
         -4.0622e+00, -3.4219e+00],
        ...,
        [ 6.2441e+02,  5.8991e+02,  6.3266e+02,  ...,  6.3929e+02,
          7.6063e+02,  8.4888e+02],
        [ 2.6323e+03,  3.1801e+03,  3.6581e+03,  ...,  6.1361e+03,
          6.2719e+03,  6.3632e+03],
        [-3.8966e-02, -3.8966e-02, -3.8966e-02,  ...,  2.9033e-02,
          2.9033e-02,  2.9033e-02]]), tensor([1., 9.]))


In [3]:
model_modules.do_gpu = torch.cuda.is_available()
# model_modules.do_gpu = False

if(model_modules.do_gpu):
    print(torch.cuda.get_device_name())
else:
    print('using CPU')

data_loader = DataLoader(data_train,batch_size = 128,shuffle=True,pin_memory=model_modules.do_gpu)
data_loader_test  = DataLoader(data_test,batch_size = 128,shuffle=True,pin_memory=model_modules.do_gpu)

GeForce GTX 1070 Ti


In [4]:
# input_size = num EEG channels


input_size = 40
torch.manual_seed(55555)


save_folder = r'w_all\v001'

model = LSTM2CNN_2(input_size=input_size,number_layers=1)

'''
    load w
    
'''
load_w_path = r'\w_all\v001\model_002_7dot83'
# load_w_path = None



if(load_w_path is not None):
    model.load_state_dict(torch.load(r'C:\Users\rakna\Desktop\AIT_working\AIT_CP_Project\this_folder_git_ignore'+load_w_path))

best_loss = None
epoch = -1
if(load_w_path is not None):
    name = load_w_path.split('\\')[-1]
    epoch = int(name.split('_')[-2])
    best_loss = float(name.split('_')[-1].replace('dot','.'))

print(f"continue train from epoch {epoch} best_loss = {best_loss}")


'''
    tranfer learning
'''


# load_w_path = r'\w\v028\model_036_0dot90'
# load_w_path = r'\w_s12\v002\model_013_4dot29'
# model.load_state_dict(torch.load(r'C:\Users\rakna\Desktop\AIT_working\AIT_CP_Project\this_folder_git_ignore'+load_w_path))

# epoch => what's epoch now


model = move_to_gpu(model)
# print(model.eval())
loss_func = nn.MSELoss()



# lr_rate = 5e-8
# lr_rate = 5e-7
lr_rate = 5e-6
# lr_rate = 5e-5
# lr_rate = 5e-4
# lr_rate = 5e-3
# lr_rate = 1e-3



l2_rate = 1e-4
# l2_rate = 8e-3
# l2_rate = 1e-3

optimizer = torch.optim.Adam(model.parameters_with_l2(),lr=lr_rate,weight_decay=l2_rate)
optimizer_without_l2 = torch.optim.Adam(model.parameters_without_l2(),lr=lr_rate)


continue train from epoch 2 best_loss = 7.83


In [ ]:
print(f"continue train from epoch {epoch} best_loss = {best_loss}")

pl_1 = Limit(1)
pl_1_2 = Limit(1)

do_train = True
is_limit = False

def train(not_ok=0):
    pl = Limit(3)
    loss_train_batch = 0
    c=0
    for train_data,label in tqdm(data_loader):

        if(is_limit):
            if(CounterBreak.set_count(2).count()):
                pl_1('limit train_data')
                break

        train_data = move_to_gpu(train_data)
        label = move_to_gpu(label)


        optimizer.zero_grad()
        optimizer_without_l2.zero_grad()
        bz = train_data.shape[0]
        model.reset_hidden(bz)


        if(not_ok==0):
            y_hat = (model(train_data))
        else:
            model(1)

        loss = loss_func(y_hat,label)
        loss_train_batch+=float(loss)
        c+=1
        loss.backward()
        optimizer.step()
        optimizer_without_l2.step()

#         del_gpu(train_data,label)

    log(loss_train_batch/c,title=f"loss train at epoch {epoch}")
def model_eval():
    '''
                find loss test
    '''
    global best_loss
    loss_test=0
    c = 0
    with torch.no_grad():
        for test_data,y_test in tqdm(data_loader_test):


            if(is_limit):
                if(CounterBreak.set_count(2).count()):
                    pl_1_2('limit test_data')
                    break

            test_data = move_to_gpu(test_data)    
            y_test = move_to_gpu(y_test)

            bz = test_data.shape[0]

            model.reset_hidden(bz)
            y_hat_test = model(test_data)

            batch_loss_test = loss_func(y_hat_test,y_test)

            loss_test+= float(batch_loss_test)
            c+=1

#             del_gpu(test_data,y_test)


    loss_test = loss_test/c
    log(loss_test,title=f"loss_test at epoch {epoch}")


    rp = r"C:\Users\rakna\Desktop\AIT_working\AIT_CP_Project\this_folder_git_ignore"
    if(CounterMod.init().set_mod(5).count()):
        log(loss_test,title=f"(from CounterMod)loss test at epoch {epoch}")
        log('ram gpu',torch.cuda.memory_allocated())
        torch.save(model.state_dict(), os.path.join(rp,save_folder,f'./autosave_model_{epoch:03}_{"{:.2f}".format(loss_test).replace(".","dot") }' ) )

    if(best_loss is None or loss_test<best_loss):
        best_loss=loss_test

        torch.save(model.state_dict(), os.path.join(rp,save_folder,f'./model_{epoch:03}_{"{:.2f}".format(loss_test).replace(".","dot") }' ) )
        log(loss_test,title=f"loss test at epoch {epoch}")
        log(title='save w',n=60)
        playsound(os.path.join(sound_path,'got_it.mp3'))
    else:
        playsound(os.path.join(sound_path,'trying_best.mp3'))

must_check =False
if(do_train):
    while True:
        try:
            epoch+=1    
            if(must_check):
                print("save check")
                model_eval()
                must_check=False
            train()
            model_eval()
    

        except RuntimeError as e:
            epoch-=1
            playsound(os.path.join(sound_path,'help.mp3'))
            print("Help me"*10)
            must_check=True
            continue
#             raise e
# except TypeError:
#     pass
  

# except KeyboardInterrupt:
#     for sound in music_sheet2:
#         winsound.Beep(sound[0], sound[1]) 
        

        

    

    

  0%|                                                                                         | 0/6289 [00:00<?, ?it/s]

continue train from epoch 2 best_loss = 7.83


  0%|                                                                                          | 0/331 [00:00<?, ?it/s]

==============================loss train at epoch 3==============================
6.353748863803732


100%|████████████████████████████████████████████████████████████████████████████████| 331/331 [01:52<00:00,  2.95it/s]


...

  0%|                                                                                          | 0/331 [00:00<?, ?it/s]

==============================loss train at epoch 9==============================
5.12928677374806


100%|████████████████████████████████████████████████████████████████████████████████| 331/331 [01:51<00:00,  2.98it/s]


==============================loss_test at epoch 9==============================
7.240411778591191
==============================loss test at epoch 9==============================
7.240411778591191
============================================================save w============================================================


 16%|████████████▍                                                                  | 988/6289 [07:50<42:10,  2.10it/s]